Ce notebook permet de tester les résultats d'un modèle. On utilise deux indices : la RMSE et le SSIM.
Il faut bien sûr comparer les résultats entre deux modèles avec la même scène pour que les résultats soient porteurs de sens.

Le SSIM (Structure Similary Index) est un flottant dans [-1;1]. Plus il est élevé, plus les images sont similaires.

In [3]:
import path
import stats
import skimage.metrics
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

def gen_mesures(truth, inferred):
    """
    On peut personaliser les mesures que l'on souhaite.
    Cette fonction doit retourner un dictionnaire, et pour chaque clé un qui contient le score pour cette mesure.
    
    Args:
        truth (numpy.ndarray): L'image de la profondeur vérité terrain
        inferred (numpy.ndarray): L'image de la profondeur inferée

    Returns:
        Le dictionnaire qui contient les mesures.
    """
    
    ssim, diff_ssim = skimage.metrics.structural_similarity(truth, inferred, full=True)
    rmse = np.sqrt(np.mean((truth - inferred) ** 2))
    
    return {
        "ssim": ssim,
        "rmse": rmse
    }

def on_path_changed(dataset_path):
    paths = path.load_dataset_paths(dataset_path)
    df = stats.compute_dataset_stats(paths, gen_mesures)
    display(df)
    
    stats_df_data = {"mesure": [], "moyenne": [], "ecart-type": []}
    for mesure in ["ssim", "rmse"]:
        stats_df_data["mesure"].append(mesure)
        stats_df_data["moyenne"].append(df[mesure].mean())
        stats_df_data["ecart-type"].append(df[mesure].std())
    
    display(pd.DataFrame(stats_df_data))

@interact(model='C', directly_inside_dataset=True)
def on_path_changed_interact(model, directly_inside_dataset):
    
    if directly_inside_dataset:
        on_path_changed(f"/scratch_p/rafrantz/revery/hf6ag69kr0jdf8ycuw6pf8/")
    else:
        on_path_changed(f"/scratch_p/rafrantz/tf_model_revery_romeo_{model}/tests")
        

interactive(children=(Text(value='C', description='model'), Checkbox(value=True, description='directly_inside_…